In [6]:
from lexrankr import LexRank
import os
#import csv#csv
import pymysql#db


lexrank = LexRank()

##db
db = pymysql.connect(host='###', user='###', password='###' ,db='###', charset='utf8')
# prepare a cursor object using cursor() method


cursor = db.cursor()

# execute SQL query using execute() method.

cursor.execute("SELECT VERSION()")
# Fetch a single row using fetchone() method.

data = cursor.fetchone()

#db연결확인
print("Database version : %s " % data)

#csv형식의 뉴스 테이블(culture) 읽기, 읽은 후 documents 리스트에 기사 본문 저장
documents=[]
title=[]
news_id=[]

tabel_name = input("테이블 이름 입력 : ")

def make_docu(tabel_name):
    try:
        with db.cursor() as cursor:
            sql = "select * from %s where category = '정치'"%(tabel_name)
            cursor.execute("set names utf8")
            cursor.execute(sql)
            result = cursor.fetchall()
            for i in result:
                news_id.append(i[0])
                title.append(i[1])
                documents.append(i[2])
    finally:
        print("documents완료")

news_num=[]
#오픈한 csv파일 한 라인씩 읽어 테스트
def make_news():
    for i in documents:
        try:
            #print(i[1])#기사 잘 나오는지 확인
            lexrank.summarize(i)
            summaries = lexrank.probe(3)  #  (값이 '>1' 시)  문장갯수 (값이 '<1' 시) 요약비율 
            #summaries2 = lexrank.probe(.1)
            news_num.append(summaries)             
        except:
            #pass
            news_num.append(i) 
              
#컬럼 추가
#cursor.execute("ALTER TABLE culture ADD sum TEXT NOT NULL")

def save_summ(table_name):
        try:
            with db.cursor() as cursor:
                for data in range(len(news_num)):
                    #기사에 맞게 태그 집어넣기 (리스트와 sql id 비교)
                    try:
                        sql= "update %s set summarize='%s'where news_id= '%d'" % (table_name,news_num[data][0],news_id[data])
                        cursor.execute(sql) 

                    except:
                        sql= "update %s set summarize='%s'where news_id= '%d'" % (table_name,title[data],news_id[data])
                        cursor.execute(sql) 

            db.commit()
            print("요약 완료")

        finally:
            db.close
make_docu(tabel_name)
make_news()
save_summ(tabel_name)    


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


Database version : 8.0.15 
테이블 이름 입력 : news


C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


documents완료
요약 완료
